# Writing a function that finds an IBAN in a string of unrelated text

As a start project to supplement my learning in python I wanted to build a dashboard to be able to monitor my spending as well as practice my coding. 

The extract from my bank however is not very well formatted. 

Most of the information about a given payment is available in a string included at the end of each line. 

For example below is an example of a payment sent to my savings account 

Instant credit transfer in euro: ING app                                        To: MR AIDAN SMITH - BE16377470638074                                           Instantly on 25/01 - 14:23:36                                                                                                                                       
So in order to begin figuring out where my money has gone I would like to isolate and identify the IBANs in my dataset 

Below are my attempts to do this

# Getting the IBAN information

Firstly I wanted to get a list or dictionary of each IBAN and the number of characters that each countries' IBAN's contain. 

The general format of an IBAN is with the first 2 letters signifying the country and the rest of the characters will be specific to that country

For example BE17548812179621 is an IBAN from Belgium, whereas NL88ABNA5082783165 is an IBAN from the Netherlands. The Belgian IBAN has 16 characters wheareas the Dutch IBAN has 18 (all IBAN's used in this are from an IBAN randomiser so have no attachment to myself)

The tables containing this information can be found here https://www.iban.com/structure as well as on wikipedia here: https://en.wikipedia.org/wiki/International_Bank_Account_Number#IBAN_formats_by_country

After trying for a long time to scrape the wikipedia table using using BeautifulSoup I found this really easy solution by Prakhar Jhudele in this stackoverflow question https://stackoverflow.com/questions/63844703/scraping-a-table-from-multiple-table-wikipedia

This method just uses the pandas library to make a list of tables from the URL

So after accessing the table I save it as it's own dataframe and then take the first 2 characters from the IBAN Fields column as my Country Code to use for the IBAN identification

In [40]:
import pandas as pd
wikiurl = 'https://en.wikipedia.org/wiki/International_Bank_Account_Number#IBAN_formats_by_country'
tables = pd.read_html(wikiurl)

In [105]:
IBAN_df = tables[3]
IBAN_df["Country_Code"] = IBAN_df['IBAN Fields'].str[:2]
print(IBAN_df.head())

      Country  Chars BBAN Format                         IBAN Fields  \
0     Albania     28      8n,16c  ALkk bbbs sssx cccc cccc cccc cccc   
1     Andorra     24      8n,12c       ADkk bbbb ssss cccc cccc cccc   
2     Austria     20         16n            ATkk bbbb bccc cccc cccc   
3  Azerbaijan     28      4c,20n  AZkk bbbb cccc cccc cccc cccc cccc   
4     Bahrain     22      4a,14c         BHkk bbbb cccc cccc cccc cc   

                                             Comment Country_Code  
0  b = National bank code s = Branch code x = Nat...           AL  
1  b = National bank code s = Branch code c = Acc...           AD  
2          b = National bank code c = Account number           AT  
3          b = National bank code c = Account number           AZ  
4           b = National bank codec = Account number           BH  


In [106]:
iban_dict = dict(zip(IBAN_df["Country_Code"], IBAN_df["Chars"]))
print(iban_dict)

{'AL': 28, 'AD': 24, 'AT': 20, 'AZ': 28, 'BH': 22, 'BY': 28, 'BE': 16, 'BA': 20, 'BR': 29, 'BG': 22, 'CR': 22, 'HR': 21, 'CY': 28, 'CZ': 24, 'DK': 18, 'DO': 28, 'TL': 23, 'EG': 29, 'SV': 28, 'EE': 20, 'FO': 18, 'FI': 18, 'FR': 27, 'GE': 22, 'DE': 22, 'GI': 23, 'GR': 27, 'GL': 18, 'GT': 28, 'HU': 28, 'IS': 26, 'IQ': 23, 'IE': 22, 'IL': 23, 'IT': 27, 'JO': 30, 'KZ': 20, 'XK': 20, 'KW': 30, 'LV': 21, 'LB': 28, 'LY': 25, 'LI': 21, 'LT': 20, 'LU': 20, 'MK': 19, 'MT': 31, 'MR': 27, 'MU': 30, 'MC': 27, 'MD': 24, 'ME': 22, 'NL': 18, 'NO': 15, 'PK': 24, 'PS': 29, 'PL': 28, 'PT': 25, 'QA': 29, 'RO': 24, 'RU': 29, 'LC': 32, 'SM': 27, 'ST': 25, 'SA': 24, 'RS': 22, 'SC': 31, 'SK': 24, 'SI': 19, 'ES': 24, 'SD': 18, 'SE': 24, 'CH': 21, 'TN': 24, 'TR': 26, 'UA': 29, 'AE': 23, 'GB': 22, 'VA': 22, 'VG': 24}


# Finding the IBANs

Below is my slowly working towards creating a function that would print the IBANs from a test csv file

I started using my dictionary to find the length of an IBAN and then the IBAN itself in a list. 

Next I tried to use this in a string of all CAPS unrelated text which is where I ran into some identification issues.

I explain what my function does further on.

In [107]:
random_iban = [" BE17548812179621", "NL88ABNA5082783165", "DK8850515933314767", "PT75003506511855878295791", "DE92500105174818779671"]

for key in iban_dict.keys():
    for iban in random_iban:
        if iban[:2] == key:
            print(iban[:(iban_dict[key]+1)])
            print(iban_dict[key])
        

DK8850515933314767
18
DE92500105174818779671
22
NL88ABNA5082783165
18
PT75003506511855878295791
25


In [108]:
random_string = "THERE IS AN ACCOUNT NL88ABNA5082783165 IN THIS STRING BUT IT MAY BE ALL FOR NOTHING"

letter = 'BE'
position = random_string.rfind(letter)

print(random_string[position: (position + 16)])

BE ALL FOR NOTHI


In [109]:
for key in iban_dict.keys():
    if key in random_string:
        position = random_string.rfind(key)
        
            end_iban = position +((iban_dict[key]+1))
            print(random_string[ position: end_iban])

NL88ABNA5082783165 


In [282]:
test_csv = "C:\\Users\\Aidan\\Documents\\Jupyter\\Iban Project\\Test.csv"
test_df = pd.read_csv(test_csv)
print(test_df)

                                             Message
0                              There is nothing here
1  Instant credit transfer in euro: ING app      ...
2  Instant credit transfer in euro: ING app      ...
3  Instant credit transfer in euro: ING app      ...
4  Instant credit transfer in euro: ING app      ...
5  Instant credit transfer in euro: ING app      ...
6  Instant credit transfer in euro: ING app      ...
7  Instant credit transfer in euro: ING app      ...
8  Instant credit transfer in euro: ING app      ...
9  Instant credit transfer in euro: ING app      ...


In [117]:
for i,row in test_df.iterrows():
    for key in iban_dict.keys():
        if key in row["Entry Details"]:
            position = row["Entry Details"].rfind(key)
            if row["Entry Details"][(position + 3)].isnumeric() == True: 
                end_iban = position +((iban_dict[key]+1))
                print(row["Entry Details"][ position: end_iban])

NL30INGB8181091612 
GB8181091612           
GR5101472149469821271692996 
BE90663896996532 
FI3695483936181936 
BE83988912945115 
BE25750687693382 
BE68631564355334 
DK2050518284255618 
DE27500105171756794451 


In [130]:
test_df["IBAN"] = ""

for i,row in test_df.iterrows():
    for key in iban_dict.keys():
        if key in row["Entry Details"]:
            position = row["Entry Details"].rfind(key)
            if row["Entry Details"][(position + 3)].isnumeric() == True: 
                end_iban = position +((iban_dict[key]+1))
                IBAN = row["Entry Details"][ position: end_iban]
                print(IBAN)
                row["IBAN"] = IBAN

NL30INGB8181091612 
GB8181091612           
GR5101472149469821271692996 
BE90663896996532 
FI3695483936181936 
BE83988912945115 
BE25750687693382 
BE68631564355334 
DK2050518284255618 
DE27500105171756794451 


In [131]:
print(test_df)

                                       Entry Details  \
0  Instant credit transfer in euro: ING app      ...   
1  Instant credit transfer in euro: ING app      ...   
2  Instant credit transfer in euro: ING app      ...   
3  Instant credit transfer in euro: ING app      ...   
4  Instant credit transfer in euro: ING app      ...   
5  Instant credit transfer in euro: ING app      ...   
6  Instant credit transfer in euro: ING app      ...   
7  Instant credit transfer in euro: ING app      ...   
8  Instant credit transfer in euro: ING app      ...   

                           IBAN  
0       GB8181091612             
1  GR5101472149469821271692996   
2             BE90663896996532   
3           FI3695483936181936   
4             BE83988912945115   
5             BE25750687693382   
6             BE68631564355334   
7           DK2050518284255618   
8       DE27500105171756794451   


In [120]:
def find_iban(dataframe, column, dictionary, new_column):
    for i,row in dataframe.iterrows():
        for key in dictionary.keys():
            if key in row[column]:
                position = row[column].rfind(key)
                if row[column][(position + 3)].isnumeric() == True: 
                    end_iban = position +((iban_dict[key]+1))
                    return row[column][position: end_iban]


# The 'Final' Function

So step by step what this function does is:

First it takes the input of the dataframe you want to look at, the specific column of strings to look at and the dictionary that contains the country codes and character limits for each IBAN from wikipedia.

It iterates over each row in the dataframe and over each row it then iterates over the dictionary. If the key (the country code) from the dictionary is found in the column of the row it saves the position as a variable.

I was originally getting a lot of non IBANS popping up as the format in the CSV from my bank includes a lot of lines that were ALL CAPS (so case sensitivity couldn't be an identifier). I noticed however that after the country code of an IBAN it is uniformly a numerical value. 

So the next check is if the third character after the position of the country code is numeric. If so it then finds the end of the IBAN using the start position and the number of characters from the dictionary (+ 1 as string slicing is non inclusive of the last number). 

The function then returns the string from the beginning position of the IBAN to the end position

I added in a counter for the number of keys found that helps me identify errors. Missing IBAN errors cannot be avoided as they are simply not included in the payment messages (in my dashboard I will have to find another way to look at these datapoints). 
Multiple IBAN found errors are going to take some further thought to iron out

In [337]:
def create_iban_column(dataframe, column, dictionary, new_column):
    dataframe[new_column] = ""
    for i,row in dataframe.iterrows():
        keyfound = 0 
        for key in dictionary.keys():
            if key in str(row[column]):
                position = row[column].rfind(key)
                if row[column][(position + 2)].isnumeric() == True:
                    keyfound +=1
                    if keyfound == 1:
                        end_iban = position +((iban_dict[key]+1))
                        IBAN = row[column][ position: end_iban]
                        #print(IBAN)
                        row[new_column] = IBAN
                        #print(row[new_column])
            if keyfound > 1:
                row[new_column] = "Error in message: more than one IBAN recognised"
            if keyfound == 0:
                    row[new_column] = "Error in message: no IBAN found"
        #print(keyfound)
            

In [341]:
test_csv = "C:\\Users\\Aidan\\Documents\\Jupyter\\Iban Project\\Test.csv"
test_df = pd.read_csv(test_csv)
print(test_df)

                                             Message
0                              There is nothing here
1  Instant credit transfer in euro: ING app      ...
2  Instant credit transfer in euro: ING app      ...
3  Instant credit transfer in euro: ING app      ...
4  Instant credit transfer in euro: ING app      ...
5  Instant credit transfer in euro: ING app      ...
6  Instant credit transfer in euro: ING app      ...
7  Instant credit transfer in euro: ING app      ...
8  Instant credit transfer in euro: ING app      ...
9  Instant credit transfer in euro: ING app      ...


In [342]:
create_iban_column(test_df,"Message", iban_dict, "IBAN")
test_df.to_csv(r"C:\\Users\\Aidan\\Documents\\Jupyter\\Iban Project\\testdatawithIBAN.csv")


In [343]:
test_df.head()

,Message,IBAN
0,There is nothing here,Error in message: no IBAN found
1,Instant credit transfer in euro: ING app ...,Error in message: more than one IBAN recognised
2,Instant credit transfer in euro: ING app ...,GR5101472149469821271692996
3,Instant credit transfer in euro: ING app ...,BE90663896996532
4,Instant credit transfer in euro: ING app ...,FI3695483936181936
